<a href="https://colab.research.google.com/github/anishdakshesh/Fiserv_Python/blob/main/Assignment_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Define custom exception class
class ValidationException(Exception):
    pass

# Define the function to validate the file
def validate_file(filename):
    try:
        with open(filename, 'r') as file:
            next(file)  # Skip header line
            for line in file:
                car_id, miles = line.strip().split(',')
                car_id = car_id.strip()
                miles = miles.strip()

                # Check if miles is an integer
                if not miles.isdigit():
                    raise ValidationException(f"Invalid mileage:  {miles}")

    except FileNotFoundError:
        raise ValidationException(f"File not found: {filename}")

# Example usage
def ex1():
    try:
        validate_file("/content/input.txt")
    except ValidationException as ve:
        print(ve)

# Run example
ex1()


Invalid mileage:  32.13


In [7]:
import csv
import os

def find_total_visits():
    total_visits = 0
    filenames = ["/content/week-1.csv", "/content/week-2.csv", "/content/week-3.csv"]

    for filename in filenames:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header
                for row in reader:
                    total_visits += sum(int(x.strip()) for x in row[1:])  # Sum all daily values

    return total_visits

def ex2():
    total = find_total_visits()
    print(f"Total visits: {total}.")

# Run example
ex2()


Total visits: 21.


In [8]:
import re
import os

def count_words(filename="/content/words.txt"):
    # Check if input file exists
    if not os.path.exists(filename):
        print(f"File not found: {filename}")
        return 0

    small_words = set()
    large_words = set()

    # Read the file
    with open(filename, 'r') as file:
        text = file.read()

    # Extract words ignoring punctuation
    words = re.findall(r"\b\w+\b", text)

    # Separate into small and large
    for word in words:
        if len(word) < 3:
            small_words.add(word)
        else:
            large_words.add(word)

    # Create output files automatically
    with open("small-words.txt", "w") as small_file:
        for word in sorted(small_words):
            small_file.write(word + "\n")

    with open("large-words.txt", "w") as large_file:
        for word in sorted(large_words):
            large_file.write(word + "\n")

    # Return count of unique words
    return len(small_words.union(large_words))


# Usage example
def ex3():
    total_words = count_words("words.txt")
    print(f"Total words: {total_words}.")

ex3()


Total words: 16.


In [10]:
%pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.1 MB/s eta 0:00:00


In [13]:
import boto3

def calculate():
    log = []
    student_id = "123456"  # 🔹 Replace with your actual student ID
    log_filename = f"calculator-log-{student_id}.txt"

    while True:
        first = input("Enter first number: ")
        if first.lower() == 'q':
            break

        second = input("Enter second number: ")
        if second.lower() == 'q':
            break

        try:
            num1 = float(first)
            num2 = float(second)
            result = num1 + num2
            print(f"{num1} + {num2} = {result}")
            log.append(f"{num1} + {num2} = {result}")
        except ValueError:
            print("Invalid input. Please enter a number or 'q' to quit.")

    # Write log to file
    with open(log_filename, 'w') as f:
        for entry in log:
            f.write(entry + "\n")

    # Upload to S3
    try:
        s3 = boto3.client('s3')
        bucket_name = "your-s3-bucket-name"  # 🔹 Replace with your S3 bucket
        s3.upload_file(log_filename, bucket_name, log_filename)
        print("*** Uploaded to S3 ***")
    except Exception as e:
        print(f"Error uploading to S3: {e}")


# Usage example
def ex4():
    calculate()
ex4()


Enter first number: 5
Enter second number: 9
5.0 + 9.0 = 14.0
Enter first number: 1
Enter second number: 2
1.0 + 2.0 = 3.0
Enter first number: 3
Enter second number: 4
3.0 + 4.0 = 7.0
Enter first number: h
Enter second number: g
Invalid input. Please enter a number or 'q' to quit.
Enter first number: q
Error uploading to S3: Unable to locate credentials


In [14]:
import csv
from pprint import pprint

def build_car_list():
    car_data = []
    cars = {}

    # Step 1: Read car IDs and models
    with open("/content/car-ids.txt", 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            car_id = int(row[0].strip())
            model = row[1].strip()
            cars[car_id] = model

    # Step 2: Read mileage data
    with open("/content/input.txt", 'r') as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        for row in reader:
            car_id = int(row[0].strip())
            miles_str = row[1].strip()
            try:
                miles = int(miles_str)
                # Step 3: Combine valid entries
                car_data.append({
                    "id": car_id,
                    "miles": miles,
                    "model": cars.get(car_id, "Unknown")
                })
            except ValueError:
                # Skip invalid mileage (like 32.13)
                continue

    return car_data


def ex5():
    car_list = build_car_list()
    pprint(car_list)


# Run example
ex5()


[{'id': 1, 'miles': 10111, 'model': 'Ford'},
 {'id': 2, 'miles': 30333, 'model': 'Chevy'},
 {'id': 3, 'miles': 40443, 'model': 'Toyota'},
 {'id': 5, 'miles': 60000, 'model': 'Nissan'}]
